In [5]:
import random
from graph_tool import Graph, draw
from graph_tool.topology import min_spanning_tree, shortest_distance
import python_codes.files_operators

filename1 = "Germany50"

# 读取网络数据
read_graph, read_pos = python_codes.files_operators.read_files(f"../networks_clusters/{filename1}.net")
print("nodes: ", read_graph.num_vertices(), "links ", read_graph.num_edges())

# 参数设置
POP_SIZE = 50
NUM_GENERATIONS = 100
MUTATION_RATE = 0.1
TOURNAMENT_SIZE = 3

# 初始化种群
def initialize_population(graph, pop_size):
    population = []
    for _ in range(pop_size):
        mst = min_spanning_tree(graph)
        mst_graph = Graph(directed=False)
        mst_graph.add_vertex(graph.num_vertices())
        for edge, in_mst in zip(graph.edges(), mst):
            if in_mst:
                mst_graph.add_edge(edge.source(), edge.target())
        # 复制顶点和边属性
        for prop_name, prop_map in graph.vertex_properties.items():
            mst_graph.vertex_properties[prop_name] = mst_graph.new_vertex_property(prop_map.value_type())
            for v in mst_graph.vertices():
                mst_graph.vertex_properties[prop_name][v] = graph.vertex_properties[prop_name][v]
        for prop_name, prop_map in graph.edge_properties.items():
            mst_graph.edge_properties[prop_name] = mst_graph.new_edge_property(prop_map.value_type())
            for e in mst_graph.edges():
                mst_graph.edge_properties[prop_name][e] = graph.edge_properties[prop_name][graph.edge(e.source(), e.target())]
        population.append(mst_graph)
    return population

# 适应度函数：计算总路径长度
def fitness(graph):
    total_length = 0
    for v in graph.vertices():
        dist_map = shortest_distance(graph, source=v)
        total_length += sum(dist_map.a)
    return total_length

# 选择父代
def tournament_selection(population, k):
    selected = random.sample(population, k)
    selected.sort(key=fitness)
    return selected[0]

# 交叉操作
def crossover(parent1, parent2):
    size = parent1.num_edges()
    edges1 = list(parent1.edges())
    edges2 = list(parent2.edges())
    child = Graph(directed=False)
    child.add_vertex(parent1.num_vertices())
    for i in range(size // 2):
        edge = edges1[i]
        child.add_edge(edge.source(), edge.target())
    for i in range(size // 2, size):
        edge = edges2[i]
        child.add_edge(edge.source(), edge.target())
    # 复制顶点和边属性
    for prop_name, prop_map in parent1.vertex_properties.items():
        child.vertex_properties[prop_name] = child.new_vertex_property(prop_map.value_type())
        for v in child.vertices():
            child.vertex_properties[prop_name][v] = parent1.vertex_properties[prop_name][v]
    for prop_name, prop_map in parent1.edge_properties.items():
        child.edge_properties[prop_name] = child.new_edge_property(prop_map.value_type())
        for e in child.edges():
            child.edge_properties[prop_name][e] = parent1.edge_properties[prop_name][parent1.edge(e.source(), e.target())]
    return child

# 变异操作
def mutate(graph, mutation_rate):
    if random.random() < mutation_rate:
        edges = list(graph.edges())
        graph.remove_edge(random.choice(edges))
        available_edges = list(read_graph.edges())
        new_edge = random.choice(available_edges)
        graph.add_edge(new_edge.source(), new_edge.target())

# 主遗传算法
def genetic_algorithm(graph, pop_size, num_generations, mutation_rate, tournament_size):
    population = initialize_population(graph, pop_size)
    best_individual = None
    best_fitness = float('inf')

    for generation in range(num_generations):
        new_population = []
        for _ in range(pop_size // 2):
            parent1 = tournament_selection(population, tournament_size)
            parent2 = tournament_selection(population, tournament_size)
            child1 = crossover(parent1, parent2)
            child2 = crossover(parent2, parent1)
            mutate(child1, mutation_rate)
            mutate(child2, mutation_rate)
            new_population.extend([child1, child2])

        population = new_population

        # 评估新种群
        for individual in population:
            ind_fitness = fitness(individual)
            if ind_fitness < best_fitness:
                best_fitness = ind_fitness
                best_individual = individual

        print(f"Generation {generation}: Best fitness = {best_fitness}")

    return best_individual

# 执行遗传算法
best_mst = genetic_algorithm(read_graph, POP_SIZE, NUM_GENERATIONS, MUTATION_RATE, TOURNAMENT_SIZE)

# 打印最优解的节点和边数量
print("Best MST nodes: ", best_mst.num_vertices(), "Best MST links: ", best_mst.num_edges())

# 绘制最优解
draw.graph_draw(best_mst, read_pos, vertex_text=best_mst.vertex_properties["number"], edge_color='blue', output_size=(1000, 1000), output=f'./output_GA/{filename1}_{NUM_GENERATIONS}_best_mst.pdf')

nodes:  50 links  69
Generation 0: Best fitness = 20368
Generation 1: Best fitness = 20368
Generation 2: Best fitness = 20008
Generation 3: Best fitness = 20008
Generation 4: Best fitness = 19824
Generation 5: Best fitness = 19824
Generation 6: Best fitness = 19824
Generation 7: Best fitness = 19824
Generation 8: Best fitness = 19824
Generation 9: Best fitness = 19824
Generation 10: Best fitness = 19824
Generation 11: Best fitness = 19824
Generation 12: Best fitness = 19824
Generation 13: Best fitness = 19824
Generation 14: Best fitness = 19824
Generation 15: Best fitness = 19824
Generation 16: Best fitness = 19824
Generation 17: Best fitness = 19824
Generation 18: Best fitness = 19824
Generation 19: Best fitness = 19824
Generation 20: Best fitness = 19824
Generation 21: Best fitness = 19824
Generation 22: Best fitness = 19570
Generation 23: Best fitness = 19570
Generation 24: Best fitness = 19570
Generation 25: Best fitness = 19570
Generation 26: Best fitness = 19570
Generation 27: Be

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7c217652e690, at 0x7c2204fc7140>

In [6]:
import random
from graph_tool import Graph, draw
from graph_tool.topology import min_spanning_tree, shortest_distance
import python_codes.files_operators

filename1 = "Germany50"

# 读取网络数据
read_graph, read_pos = python_codes.files_operators.read_files(f"../networks_clusters/{filename1}.net")
print("nodes: ", read_graph.num_vertices(), "links ", read_graph.num_edges())

# 参数设置
POP_SIZE = 50
NUM_GENERATIONS = 200
MUTATION_RATE = 0.1
TOURNAMENT_SIZE = 3

# 初始化种群
def initialize_population(graph, pop_size):
    population = []
    for _ in range(pop_size):
        mst = min_spanning_tree(graph)
        mst_graph = Graph(directed=False)
        mst_graph.add_vertex(graph.num_vertices())
        for edge, in_mst in zip(graph.edges(), mst):
            if in_mst:
                mst_graph.add_edge(edge.source(), edge.target())
        # 复制顶点和边属性
        for prop_name, prop_map in graph.vertex_properties.items():
            mst_graph.vertex_properties[prop_name] = mst_graph.new_vertex_property(prop_map.value_type())
            for v in mst_graph.vertices():
                mst_graph.vertex_properties[prop_name][v] = graph.vertex_properties[prop_name][v]
        for prop_name, prop_map in graph.edge_properties.items():
            mst_graph.edge_properties[prop_name] = mst_graph.new_edge_property(prop_map.value_type())
            for e in mst_graph.edges():
                mst_graph.edge_properties[prop_name][e] = graph.edge_properties[prop_name][graph.edge(e.source(), e.target())]
        population.append(mst_graph)
    return population

# 适应度函数：计算总路径长度
def fitness(graph):
    total_length = 0
    for v in graph.vertices():
        dist_map = shortest_distance(graph, source=v)
        total_length += sum(dist_map.a)
    return total_length

# 选择父代
def tournament_selection(population, k):
    selected = random.sample(population, k)
    selected.sort(key=fitness)
    return selected[0]

# 交叉操作
def crossover(parent1, parent2):
    size = parent1.num_edges()
    edges1 = list(parent1.edges())
    edges2 = list(parent2.edges())
    child = Graph(directed=False)
    child.add_vertex(parent1.num_vertices())
    for i in range(size // 2):
        edge = edges1[i]
        child.add_edge(edge.source(), edge.target())
    for i in range(size // 2, size):
        edge = edges2[i]
        child.add_edge(edge.source(), edge.target())
    # 复制顶点和边属性
    for prop_name, prop_map in parent1.vertex_properties.items():
        child.vertex_properties[prop_name] = child.new_vertex_property(prop_map.value_type())
        for v in child.vertices():
            child.vertex_properties[prop_name][v] = parent1.vertex_properties[prop_name][v]
    for prop_name, prop_map in parent1.edge_properties.items():
        child.edge_properties[prop_name] = child.new_edge_property(prop_map.value_type())
        for e in child.edges():
            child.edge_properties[prop_name][e] = parent1.edge_properties[prop_name][parent1.edge(e.source(), e.target())]
    return child

# 变异操作
def mutate(graph, mutation_rate):
    if random.random() < mutation_rate:
        edges = list(graph.edges())
        graph.remove_edge(random.choice(edges))
        available_edges = list(read_graph.edges())
        new_edge = random.choice(available_edges)
        graph.add_edge(new_edge.source(), new_edge.target())

# 主遗传算法
def genetic_algorithm(graph, pop_size, num_generations, mutation_rate, tournament_size):
    population = initialize_population(graph, pop_size)
    best_individual = None
    best_fitness = float('inf')

    for generation in range(num_generations):
        new_population = []
        for _ in range(pop_size // 2):
            parent1 = tournament_selection(population, tournament_size)
            parent2 = tournament_selection(population, tournament_size)
            child1 = crossover(parent1, parent2)
            child2 = crossover(parent2, parent1)
            mutate(child1, mutation_rate)
            mutate(child2, mutation_rate)
            new_population.extend([child1, child2])

        population = new_population

        # 评估新种群
        for individual in population:
            ind_fitness = fitness(individual)
            if ind_fitness < best_fitness:
                best_fitness = ind_fitness
                best_individual = individual

        print(f"Generation {generation}: Best fitness = {best_fitness}")

    return best_individual

# 执行遗传算法
best_mst = genetic_algorithm(read_graph, POP_SIZE, NUM_GENERATIONS, MUTATION_RATE, TOURNAMENT_SIZE)

# 打印最优解的节点和边数量
print("Best MST nodes: ", best_mst.num_vertices(), "Best MST links: ", best_mst.num_edges())

# 绘制最优解
draw.graph_draw(best_mst, read_pos, vertex_text=best_mst.vertex_properties["number"], edge_color='blue', output_size=(1000, 1000), output=f'./output_GA/{filename1}_{NUM_GENERATIONS}_best_mst.pdf')


nodes:  50 links  69
Generation 0: Best fitness = 20368
Generation 1: Best fitness = 20368
Generation 2: Best fitness = 20368
Generation 3: Best fitness = 20368
Generation 4: Best fitness = 20368
Generation 5: Best fitness = 20368
Generation 6: Best fitness = 20368
Generation 7: Best fitness = 20368
Generation 8: Best fitness = 20368
Generation 9: Best fitness = 20368
Generation 10: Best fitness = 20368
Generation 11: Best fitness = 20368
Generation 12: Best fitness = 20360
Generation 13: Best fitness = 17678
Generation 14: Best fitness = 17678
Generation 15: Best fitness = 17678
Generation 16: Best fitness = 17678
Generation 17: Best fitness = 17678
Generation 18: Best fitness = 17678
Generation 19: Best fitness = 17678
Generation 20: Best fitness = 17678
Generation 21: Best fitness = 17678
Generation 22: Best fitness = 17678
Generation 23: Best fitness = 17678
Generation 24: Best fitness = 17678
Generation 25: Best fitness = 17678
Generation 26: Best fitness = 17678
Generation 27: Be

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7c21867e8ef0, at 0x7c2204f3bfe0>

In [7]:
import random
from graph_tool import Graph, draw
from graph_tool.topology import min_spanning_tree, shortest_distance
import python_codes.files_operators

filename1 = "Germany50"

# 读取网络数据
read_graph, read_pos = python_codes.files_operators.read_files(f"../networks_clusters/{filename1}.net")
print("nodes: ", read_graph.num_vertices(), "links ", read_graph.num_edges())

# 参数设置
POP_SIZE = 50
NUM_GENERATIONS = 300
MUTATION_RATE = 0.1
TOURNAMENT_SIZE = 3

# 初始化种群
def initialize_population(graph, pop_size):
    population = []
    for _ in range(pop_size):
        mst = min_spanning_tree(graph)
        mst_graph = Graph(directed=False)
        mst_graph.add_vertex(graph.num_vertices())
        for edge, in_mst in zip(graph.edges(), mst):
            if in_mst:
                mst_graph.add_edge(edge.source(), edge.target())
        # 复制顶点和边属性
        for prop_name, prop_map in graph.vertex_properties.items():
            mst_graph.vertex_properties[prop_name] = mst_graph.new_vertex_property(prop_map.value_type())
            for v in mst_graph.vertices():
                mst_graph.vertex_properties[prop_name][v] = graph.vertex_properties[prop_name][v]
        for prop_name, prop_map in graph.edge_properties.items():
            mst_graph.edge_properties[prop_name] = mst_graph.new_edge_property(prop_map.value_type())
            for e in mst_graph.edges():
                mst_graph.edge_properties[prop_name][e] = graph.edge_properties[prop_name][graph.edge(e.source(), e.target())]
        population.append(mst_graph)
    return population

# 适应度函数：计算总路径长度
def fitness(graph):
    total_length = 0
    for v in graph.vertices():
        dist_map = shortest_distance(graph, source=v)
        total_length += sum(dist_map.a)
    return total_length

# 选择父代
def tournament_selection(population, k):
    selected = random.sample(population, k)
    selected.sort(key=fitness)
    return selected[0]

# 交叉操作
def crossover(parent1, parent2):
    size = parent1.num_edges()
    edges1 = list(parent1.edges())
    edges2 = list(parent2.edges())
    child = Graph(directed=False)
    child.add_vertex(parent1.num_vertices())
    for i in range(size // 2):
        edge = edges1[i]
        child.add_edge(edge.source(), edge.target())
    for i in range(size // 2, size):
        edge = edges2[i]
        child.add_edge(edge.source(), edge.target())
    # 复制顶点和边属性
    for prop_name, prop_map in parent1.vertex_properties.items():
        child.vertex_properties[prop_name] = child.new_vertex_property(prop_map.value_type())
        for v in child.vertices():
            child.vertex_properties[prop_name][v] = parent1.vertex_properties[prop_name][v]
    for prop_name, prop_map in parent1.edge_properties.items():
        child.edge_properties[prop_name] = child.new_edge_property(prop_map.value_type())
        for e in child.edges():
            child.edge_properties[prop_name][e] = parent1.edge_properties[prop_name][parent1.edge(e.source(), e.target())]
    return child

# 变异操作
def mutate(graph, mutation_rate):
    if random.random() < mutation_rate:
        edges = list(graph.edges())
        graph.remove_edge(random.choice(edges))
        available_edges = list(read_graph.edges())
        new_edge = random.choice(available_edges)
        graph.add_edge(new_edge.source(), new_edge.target())

# 主遗传算法
def genetic_algorithm(graph, pop_size, num_generations, mutation_rate, tournament_size):
    population = initialize_population(graph, pop_size)
    best_individual = None
    best_fitness = float('inf')

    for generation in range(num_generations):
        new_population = []
        for _ in range(pop_size // 2):
            parent1 = tournament_selection(population, tournament_size)
            parent2 = tournament_selection(population, tournament_size)
            child1 = crossover(parent1, parent2)
            child2 = crossover(parent2, parent1)
            mutate(child1, mutation_rate)
            mutate(child2, mutation_rate)
            new_population.extend([child1, child2])

        population = new_population

        # 评估新种群
        for individual in population:
            ind_fitness = fitness(individual)
            if ind_fitness < best_fitness:
                best_fitness = ind_fitness
                best_individual = individual

        print(f"Generation {generation}: Best fitness = {best_fitness}")

    return best_individual

# 执行遗传算法
best_mst = genetic_algorithm(read_graph, POP_SIZE, NUM_GENERATIONS, MUTATION_RATE, TOURNAMENT_SIZE)

# 打印最优解的节点和边数量
print("Best MST nodes: ", best_mst.num_vertices(), "Best MST links: ", best_mst.num_edges())

# 绘制最优解
draw.graph_draw(best_mst, read_pos, vertex_text=best_mst.vertex_properties["number"], edge_color='blue', output_size=(1000, 1000), output=f'./output_GA/{filename1}_{NUM_GENERATIONS}_best_mst.pdf')

nodes:  50 links  69
Generation 0: Best fitness = 20368
Generation 1: Best fitness = 20368
Generation 2: Best fitness = 20368
Generation 3: Best fitness = 20368
Generation 4: Best fitness = 20368
Generation 5: Best fitness = 20368
Generation 6: Best fitness = 20368
Generation 7: Best fitness = 20368
Generation 8: Best fitness = 20368
Generation 9: Best fitness = 20368
Generation 10: Best fitness = 20368
Generation 11: Best fitness = 20368
Generation 12: Best fitness = 20368
Generation 13: Best fitness = 20368
Generation 14: Best fitness = 20368
Generation 15: Best fitness = 20368
Generation 16: Best fitness = 19440
Generation 17: Best fitness = 19440
Generation 18: Best fitness = 19440
Generation 19: Best fitness = 19440
Generation 20: Best fitness = 19440
Generation 21: Best fitness = 19440
Generation 22: Best fitness = 19440
Generation 23: Best fitness = 19440
Generation 24: Best fitness = 19440
Generation 25: Best fitness = 19440
Generation 26: Best fitness = 19440
Generation 27: Be

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7c21841e18e0, at 0x7c2187ab6270>

In [8]:
import random
from graph_tool import Graph, draw
from graph_tool.topology import min_spanning_tree, shortest_distance
import python_codes.files_operators

filename1 = "Germany50"

# 读取网络数据
read_graph, read_pos = python_codes.files_operators.read_files(f"../networks_clusters/{filename1}.net")
print("nodes: ", read_graph.num_vertices(), "links ", read_graph.num_edges())

# 参数设置
POP_SIZE = 50
NUM_GENERATIONS = 500
MUTATION_RATE = 0.1
TOURNAMENT_SIZE = 3

# 初始化种群
def initialize_population(graph, pop_size):
    population = []
    for _ in range(pop_size):
        mst = min_spanning_tree(graph)
        mst_graph = Graph(directed=False)
        mst_graph.add_vertex(graph.num_vertices())
        for edge, in_mst in zip(graph.edges(), mst):
            if in_mst:
                mst_graph.add_edge(edge.source(), edge.target())
        # 复制顶点和边属性
        for prop_name, prop_map in graph.vertex_properties.items():
            mst_graph.vertex_properties[prop_name] = mst_graph.new_vertex_property(prop_map.value_type())
            for v in mst_graph.vertices():
                mst_graph.vertex_properties[prop_name][v] = graph.vertex_properties[prop_name][v]
        for prop_name, prop_map in graph.edge_properties.items():
            mst_graph.edge_properties[prop_name] = mst_graph.new_edge_property(prop_map.value_type())
            for e in mst_graph.edges():
                mst_graph.edge_properties[prop_name][e] = graph.edge_properties[prop_name][graph.edge(e.source(), e.target())]
        population.append(mst_graph)
    return population

# 适应度函数：计算总路径长度
def fitness(graph):
    total_length = 0
    for v in graph.vertices():
        dist_map = shortest_distance(graph, source=v)
        total_length += sum(dist_map.a)
    return total_length

# 选择父代
def tournament_selection(population, k):
    selected = random.sample(population, k)
    selected.sort(key=fitness)
    return selected[0]

# 交叉操作
def crossover(parent1, parent2):
    size = parent1.num_edges()
    edges1 = list(parent1.edges())
    edges2 = list(parent2.edges())
    child = Graph(directed=False)
    child.add_vertex(parent1.num_vertices())
    for i in range(size // 2):
        edge = edges1[i]
        child.add_edge(edge.source(), edge.target())
    for i in range(size // 2, size):
        edge = edges2[i]
        child.add_edge(edge.source(), edge.target())
    # 复制顶点和边属性
    for prop_name, prop_map in parent1.vertex_properties.items():
        child.vertex_properties[prop_name] = child.new_vertex_property(prop_map.value_type())
        for v in child.vertices():
            child.vertex_properties[prop_name][v] = parent1.vertex_properties[prop_name][v]
    for prop_name, prop_map in parent1.edge_properties.items():
        child.edge_properties[prop_name] = child.new_edge_property(prop_map.value_type())
        for e in child.edges():
            child.edge_properties[prop_name][e] = parent1.edge_properties[prop_name][parent1.edge(e.source(), e.target())]
    return child

# 变异操作
def mutate(graph, mutation_rate):
    if random.random() < mutation_rate:
        edges = list(graph.edges())
        graph.remove_edge(random.choice(edges))
        available_edges = list(read_graph.edges())
        new_edge = random.choice(available_edges)
        graph.add_edge(new_edge.source(), new_edge.target())

# 主遗传算法
def genetic_algorithm(graph, pop_size, num_generations, mutation_rate, tournament_size):
    population = initialize_population(graph, pop_size)
    best_individual = None
    best_fitness = float('inf')

    for generation in range(num_generations):
        new_population = []
        for _ in range(pop_size // 2):
            parent1 = tournament_selection(population, tournament_size)
            parent2 = tournament_selection(population, tournament_size)
            child1 = crossover(parent1, parent2)
            child2 = crossover(parent2, parent1)
            mutate(child1, mutation_rate)
            mutate(child2, mutation_rate)
            new_population.extend([child1, child2])

        population = new_population

        # 评估新种群
        for individual in population:
            ind_fitness = fitness(individual)
            if ind_fitness < best_fitness:
                best_fitness = ind_fitness
                best_individual = individual

        print(f"Generation {generation}: Best fitness = {best_fitness}")

    return best_individual

# 执行遗传算法
best_mst = genetic_algorithm(read_graph, POP_SIZE, NUM_GENERATIONS, MUTATION_RATE, TOURNAMENT_SIZE)

# 打印最优解的节点和边数量
print("Best MST nodes: ", best_mst.num_vertices(), "Best MST links: ", best_mst.num_edges())

# 绘制最优解
draw.graph_draw(best_mst, read_pos, vertex_text=best_mst.vertex_properties["number"], edge_color='blue', output_size=(1000, 1000), output=f'./output_GA/{filename1}_{NUM_GENERATIONS}_best_mst.pdf')

nodes:  50 links  69
Generation 0: Best fitness = 20368
Generation 1: Best fitness = 20368
Generation 2: Best fitness = 20368
Generation 3: Best fitness = 20368
Generation 4: Best fitness = 20368
Generation 5: Best fitness = 20368
Generation 6: Best fitness = 20368
Generation 7: Best fitness = 20368
Generation 8: Best fitness = 20368
Generation 9: Best fitness = 20368
Generation 10: Best fitness = 20368
Generation 11: Best fitness = 20368
Generation 12: Best fitness = 20368
Generation 13: Best fitness = 18128
Generation 14: Best fitness = 18128
Generation 15: Best fitness = 18128
Generation 16: Best fitness = 18128
Generation 17: Best fitness = 18128
Generation 18: Best fitness = 18128
Generation 19: Best fitness = 18128
Generation 20: Best fitness = 18128
Generation 21: Best fitness = 18128
Generation 22: Best fitness = 18128
Generation 23: Best fitness = 18128
Generation 24: Best fitness = 18128
Generation 25: Best fitness = 18128
Generation 26: Best fitness = 18128
Generation 27: Be

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7c21867de9f0, at 0x7c218676d580>